In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
from static_grader import grader

# Spark Miniproject


Stack Overflow is a collaboratively edited question-and-answer site originally focused on programming topics. Because of the variety of features tracked, including a variety of feedback metrics, it allows for some open-ended analysis of user behavior on the site.

Stack Exchange (the parent organization) provides an anonymized [data dump](https://archive.org/details/stackexchange), and we'll use Spark to perform data manipulation, analysis, and machine learning on this data set. As a side note, there's also an online data explorer which allows you to query the data interactively.

*Consider*: Do we need to use Spark to work with this data set? What are our alternatives?

## Workflow


You may complete this project using the Python or Scala APIs. Most questions can be done locally, however in some cases you may want to use cloud services. See the appropriate lecture notebooks for information on how to use cloud services.

Python example:

1. Edit source code in your `main.py` file, classes in a separate `classes.py` (Class definitions need to be written in a separate file and then included at runtime.)
1. Run locally on a chunk using eg. `$SPARK_HOME/bin/spark-submit --py-files src/classes.py src/main.py data/stats results/stats/`
1. Run on GCP once your testing and development are done.

Scala example:

1. Edit source code in `Main.scala`
1. Run the command `sbt package` from the root directory of the project
1. Use spark-submit locally on a chunk: this means adding a flag like `--master local[2]` to the spark-submit command.
1. Run on GCP once your testing and development are done.

General tips:
* SBT has some nice features, for example continuous build and test, which can greatly speed up your development.
* Try `cat output_dir/* | sort -n -t , -k 1.2 -o sorted_output` to concatenate your output files, which will also be in `part-xxxxx` format.
* You can access an interactive Spark/Scala REPL with `$SPARK_HOME/bin/spark-shell`.
* You can access an interactive PySpark shell with `$SPARK_HOME/bin/pyspark`.

## Accessing the data


The data is available on S3 (s3://dataincubator-course/spark-stack-data). There are three sub-folders, `allUsers`, `allPosts`, and `allVotes`, which contain Gzipped XML with the following format:

```
<row Body="&lt;p&gt;I always validate my web pages, and I recommend you do the same BUT many large company websites DO NOT and cannot validate because the importance of the website looking exactly the same on all systems requires rules to be broken. &lt;/p&gt;&#10;&#10;&lt;p&gt;In general, valid websites help your page look good even on odd configurations (like cell phones) so you should always at least try to make it validate.&lt;/p&gt;&#10;" CommentCount="0" CreationDate="2008-10-12T20:26:29.397" Id="195995" LastActivityDate="2008-10-12T20:26:29.397" OwnerDisplayName="Eric Wendelin" OwnerUserId="25066" ParentId="195973" PostTypeId="2" Score="0" />
```

Data from the much smaller stats.stackexchange.com is available in the same format on S3 (s3://dataincubator-course/spark-stats-data). This site, Cross-Validated, will be used below in some instances to avoid working with the full data set for every question.

The full schema is available as a text file:

In [ ]:
!aws s3 cp s3://dataincubator-course/spark-stats-data/stack_exchange_schema.txt .

You can either get the data by running the appropriate S3 commands in the terminal, or by running this block for the smaller stats data set:

In [ ]:
!mkdir -p spark-stats-data
!aws s3 sync --exclude '*' --include 'all*' s3://dataincubator-course/spark-stats-data/ ./spark-stats-data
!aws s3 sync --exclude '*' --include 'posts*zip' s3://dataincubator-course/spark-stats-data/ ./spark-stats-data

And to get the much larger full data set (be warned, this can take 20 or more minutes, so you may want to run it in the terminal to avoid locking up the notebook):

In [ ]:
!mkdir -p spark-stack-data
!aws s3 sync --exclude '*' --include 'all*' s3://dataincubator-course/spark-stack-data/ ./spark-stack-data

## Data input and parsing


Some rows are split across multiple lines; these can be discarded. Incorrectly formatted XML can also be ignored. It is enough to simply skip problematic rows, the loss of data will not significantly impact our results on this large data sets.

You will need to handle XML parsing yourself using the `\` selector in Scala or something like `lxml.etree` in Python. *Warning*: The built-in `xml.etree.ElementTree` behaves differently and the results don't correspond perfectly with the Scala equivalent.

To make your code more flexible, it's also recommended to incorporate command-line arguments that specify the location of the input data and where output should be written.

The goal should be to have a parsing function that can be applied to the input data to access any XML element desired. It is suggested to use a class structure so that you can create RDDs of Posts, Votes, Users, etc.

``` scala
// Command line arguments in Scala

object Main {
 def main(args: Array[String]) {
   val inputDir = args(0)
   val outputDir = args(1)
   ...
```

``` python

# Command line arguments using sysv or argparse in Python

if __name__ == '__main__':
    main(ARGS.input_dir, ARGS.output_dir)
```

Dates are parsed by default using the Long data type and Unix time (epoch time). In Java/Scala, a given timestamp represents the number of milliseconds since 1970-01-01T00:00:00Z. Also be wary of integer overflow when dealing with Longs. For example, these two are not equal:

`val year: Long = 365 * 24 * 60 * 60 * 1000`

`val year: Long = 365 * 24 * 60 * 60 * 1000L`

## Questions

## Bad XML


A simple question to test your parsing code. Create an RDD of Post objects where each Post is a valid row of XML from the Cross-Validated (stats.stackexchange.com) `allPosts` data set.

We are going to take several shortcuts to speed up and simplify our computations.  First, your parsing function to only attempt to parse rows that start with `  <row` as these denote actual data entries. This should be done in Spark as the data is being read in from disk, without any pre-Spark processing. 

Return the total number of XML rows that started with ` <row` that were subsequently **rejected** during your processing.  Note that the text is unicode, and contains non-ASCII characters.  You may need to re-encode to UTF-8 (depending on your XML parser)

Note that this cleaned data set will be used for all subsequent questions.

*Question*: Can you figure out what filters you need to put in place to avoid throwing parsing errors entirely?

In [ ]:
bad_xml = 0

grader.score('spark__bad_xml', bad_xml)

## Favorites and scores

We're interested in looking for useful patterns in the data.  If we look at the Post data again (the smaller set, `stats.stackexchange.com`), we see that many things about each post are recorded.  We're going to start by looking to see if there is a relationship between the number of times a post was favorited (the `FavoriteCount`) and the `Score`.  The score is the number of times the post was upvoted minus the number of times it was downvoted, so it is a measure of how much a post was liked.  We'd expect posts with a higher number of favorites to have better scores, since they're both measurements of how good the post is.

Let's aggregate posts by the number of favorites, and find the average score for each number of favorites.  Do this for the lowest 50 numbers of favorites.

_Note:_ Before submitting, take a look at the numbers.  Do they follow the trend you expect?

**Checkpoints**

- Total score across all posts: 299469
- Mean of first 50 favorite counts (averaging the keys themselves): 24.76

In [ ]:
favorite_score = [(0, 2.3398827696988396)]*50

grader.score('spark__favorite_score', favorite_score)

## Answer percentage


Investigate the correlation between a user's reputation and the kind of posts they make. For the 99 users with the highest reputation, single out posts which are either questions or answers and look at the percentage of these posts that are answers: *(answers / (answers + questions))*. 

Return a tuple of their **user ID** and this fraction.

You should also return (-1, fraction) to represent the case where you average over all users (so you will return 100 entries total).

Again, you only need to run this on the statistics overflow set.


#### Checkpoints

* Total questions: 52,060
* Total answers: 55,304
* Top 99 users' average reputation: 11893.464646464647

In [ ]:
answer_percentage = [(7071, 0.9107142857142857)] * 100

grader.score('spark__answer_percentage', answer_percentage)

## First question

We'd expect the first **question** a user asks to be indicative of their future behavior.  We'll dig more into that in the next problem, but for now let's see the relationship between reputation and how long it took each person to ask their first question.

For each user that asked a question, find the difference between when their account was created (`CreationDate` for the User) and when they asked their first question (`CreationDate` for their first question).  Return this time difference in days (round down, so 2.7 days counts as 2 days) for the 100 users with the highest reputation, in the form

`(UserId, Days)`

**Checkpoints**
- Users that asked a question: 23134
- Average number of days (round each user's days, then average): 30.1074258

In [ ]:
first_question = [(805, 669)] * 100

grader.score('spark__first_question', first_question)

## Identify veterans


It can be interesting to think about what factors influence a user to remain active on the site over a long period of time. In order not to bias the results towards older users, we'll define a time window between 100 and 150 days after account creation. If the user has made a post in this time, we'll consider them active and well on their way to being veterans of the site; if not, they are inactive and were likely brief users.

*Consider*: What other parameterizations of "activity" could we use, and how would they differ in terms of splitting our user base?

*Consider*: What other biases are still not dealt with, after using the above approach?

Let's see if there are differences between the first ever question posts of "veterans" vs. "brief users". For each group separately, average the score, views, number of answers, and number of favorites of the users' **first question**.

*Consider*: What story could you tell from these numbers? How do the numbers support it?


#### Checkpoints

* Total brief users: 24,864
* Total veteran users: 2,027

In [ ]:
identify_veterans = {
    "vet_score": 0,
    "vet_views": 0,
    "vet_answers": 0,
    "vet_favorites": 0,
    "brief_score": 0,
    "brief_views": 0,
    "brief_answers": 0,
    "brief_favorites": 0
}

grader.score(question_name='spark__identify_veterans', func=identify_veterans)

## Identify veterans&mdash;full


Same as above, but on the full Stack Exchange data set.

No pre-parsed data is available for this question.


#### Checkpoints

* Total brief users: 1,848,628
* Total veteran users: 288,285

In [ ]:
identify_veterans_full = {
    "vet_score": 0,
    "vet_views": 0,
    "vet_answers": 0,
    "vet_favorites": 0,
    "brief_score": 0,
    "brief_views": 0,
    "brief_answers": 0,
    "brief_favorites": 0
}

grader.score('spark__identify_veterans_full', identify_veterans_full)

## Word2vec


Word2Vec is an alternative approach for vectorizing text data. The vectorized representations of words in the vocabulary tend to be useful for predicting other words in the document, hence the famous example "vector('king') - vector('man') + vector('woman') ~= vector('queen')".

Let's see how good a Word2Vec model we can train using the tags of each Stack Exchange post as documents (this uses the full data set). Use the implementation of Word2Vec from Spark ML (this will require using DataFrames) to return a list of the top 25 closest synonyms to "ggplot2" and their similarity score in tuple format ("string", number).


#### Parameters


The dimensionality of the vector space should be 100. The random seed should be 42 in `PySpark`, 42L in Scala Spark.


#### Checkpoints

* Mean of the top 25 cosine similarities: 0.8012362027168274

In [ ]:
word2vec = [("data.frame", 0.772650957107544)] * 25

grader.score('spark__word2vec', word2vec)

## Classification


We'd like to see if we can predict the tags of a question from its body text. Instead of predicting specific tags, we will instead try to predict if a question contains one of the top ten most common tags.  

To this end, we have separated out a train and a test set from the original data.  The training and tests sets were downloaded with the stats data at the beginning of the notebook.  You can also get them from S3:
  * s3://dataincubator-course/spark-stats-data/posts_train.zip
  * s3://dataincubator-course/spark-stats-data/posts_test.zip

This will involve two steps: first, find the ten most common tags for questions in the training data set (the tags have been removed from the test set). Then train a learner to predict from the text of the question (the `Body` attribute) if it should have one of those ten tags in it - you will need to process the question text with NLP techniques such as splitting the text into tokens.

Since we can't reliably pickle Spark models, instead return a list of your predictions, sorted by the question's `Id`.  This sorting is very important, as our grader expects the results to be submitted in a particular order. These predictions should be `0` if the question isn't expected to have a tag in the top ten, and `1` if it is.

As an example, if our top tags include `spark` and `python`, and we had the following questions:

```
<row Body="..." Id="1740" Tags="<machine-learning><spark><regression>" ... />
<row Body="..." Id="723" Tags="<statistics><neurons>" ... />
<row Body="..." Id="2740" Tags="<functional><python><spark><pyspark>" ... />
```

We would expect to return `[0, 1, 1]` (for the order `[723, 1740, 2740]`).  You may need to do some format manipulation in your DataFrame to get this.

#### Checkpoints

- Number of training posts with a tag in the top 10: `19908`
- Number without: `17067`

In [ ]:
!unzip -d spark-stats-data/train spark-stats-data/posts_train.zip
!unzip -d spark-stats-data/test spark-stats-data/posts_test.zip

In [ ]:
def classification():
    return [0] * 4649

grader.score(question_name='spark__classification', func=classification)

## K-means (ungraded)


From your trained Word2Vec model, pass the vectors into a K-means clustering algorithm. Create a plot of the sum of squared error by calculating the square root of the sum of the squared distances for each point and its assigned cluster. For an independent variable use either the number of clusters k or the dimension of the Word2Vec vectorization.

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*